### Computation of the baseline

#### Load relevante moduler

In [1]:
include("../ToolBox/ToolBox.jl")
using .ToolBox
using .Geometry
using .SlcUtil
using .Load
using .Misc
using PyCall
scipy_interp = pyimport("scipy.interpolate");
ndimage = pyimport("scipy.ndimage");
rasterio = PyCall.pyimport("rasterio")

using LinearAlgebra
using Dates

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


In [2]:
"""
Load dem subset, wraps dem function.
"""
function dem_subset(dem_path, meta, view; nan_fill= 0, padding=[90,90])
    footprint = SlcUtil.footprint(meta, view)
    latlon_window = ((minimum(footprint[1]),maximum(footprint[1])),(minimum(footprint[2]),maximum(footprint[2])))
    dem = Load.dem(dem_path, latlon_window; nan_fill= 0, padding=padding);
    return dem
end

function mosaic2normal_view(mosaic_view, meta)
    @assert mosaic_view[1].start >= meta["burst_meta"]["first_line_mosaic"][1]
    @assert mosaic_view[1].stop <= (meta["burst_meta"]["first_line_mosaic"][end] + meta["lines_per_burst"])
    mosaic_view
    
    start_burst = findlast(
    meta["burst_meta"]["first_line_mosaic"].<= mosaic_view[1].start)

    lines_in_first_burst = mosaic_view[1].start -meta["burst_meta"]["first_line_mosaic"][start_burst] +1
    line_start = lines_in_first_burst + (start_burst-1)* meta["lines_per_burst"]

    end_burst = 0
    if mosaic_view[1].stop < meta["lines_per_burst"]
        end_burst = 1
    else
        end_burst = findlast(
            (meta["burst_meta"]["first_line_mosaic"] .+meta["lines_per_burst"] ).< mosaic_view[1].stop)+1
    end

    lines_in_last_burst =  mosaic_view[1].stop - meta["burst_meta"]["first_line_mosaic"][end_burst] +1
    line_end = lines_in_last_burst+ (end_burst-1)* meta["lines_per_burst"]
    return [line_start:line_end,mosaic_view[2]]
end
    
function extract_datetime(SAFE_path; start_date=true)
    extract_SAFE_name = split(SAFE_path, "/")[end]
    if start_date
        date_string = split(extract_SAFE_name, "_")[6]
    else
        date_string = split(extract_SAFE_name, "_")[7]
    end
    year = date_string[1:4]
    month = date_string[5:6]
    day = date_string[7:8]
    hour = date_string[10:11]
    minute = date_string[12:13]
    second = date_string[14:end]
    date_int = parse.(Int, [year, month, day, hour, minute, second])
    return DateTime(date_int...)
end

function days_between_acquisitions(date1, date2)
    return Dates.value(Date(date1) - Date(date2))
end

days_between_acquisitions (generic function with 1 method)

In [25]:
#input: view, dem, meta
function get_elevation(view, dem, meta, pod)
    # initilize lut
    line, sample = Misc.flatten(collect(view[1]), collect(view[2])) # line, sample

    # Get heights
    lat_dem, lon_dem, heights = Misc.flatten(dem...)
    line_sample_dem = to_line_sample(hcat(lat_dem, lon_dem), heights, pod..., meta);
    elevation = Misc.interp(line_sample_dem[:,1], line_sample_dem[:,2], heights,
                            line, sample)
    @assert sum(isnan.(elevation)) == 0
    return elevation
end

function satellite_orbit(line, meta, pod)
    azimuth_time = meta["t_start"] + (line - 1) * 1 / meta["azimuth_frequency"]
    state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(pod[1], pod[2], meta["t_start"], meta["t_stop"]);
    tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
    sat2_position, sat2_velocity = tmp[1:3], tmp[4:6]
    
    return sat2_position, sat2_velocity
end

function line_of_sight_point(line, pixel_position, meta, pod)
    sat1_position, _ = satellite_orbit(line, meta, pod)
    los = sat1_position .- pixel_position
    return los
end

function baseline_perpendicular(mosaic_view, dem, meta, pod)
    # get sample and line corresponding to view
    line1, sample1 = Misc.flatten(mosaic_view[1], mosaic_view[2])  # line, sample. Needed input structure for to_lat_lon
    
    # get elevation at view
    height = get_elevation(mosaic_view, dem, meta[1], pod[1])
    
    # compute lat, lon from line sample and height
    lat, lon = Geometry.to_lat_lon(hcat(line1, sample1), height, pod[1][1], pod[1][2], meta[1])
    
    # get the position of the pixel on ground in cartesian coordinates
    pixel_position = Geometry.ellipsoid2xyz(deg2rad(lat), deg2rad(lon), height...)
    
    # compute the line of sight vector
    los1 = line_of_sight_point(line1[1], pixel_position, meta[1], pod[1])
    
    # compute the 2nd line of sight vector based on lat lon on ground
    line2, sample2 = to_line_sample(hcat(lat, lon), height, pod[2][1], pod[2][2], meta[2])
    los2 = line_of_sight_point(line2[1], pixel_position, meta[2], pod[2])
    
    # find the correct sign
    s = sign(norm(los1) - norm(los2));
    
    # the area of the parallelogram spanned by the unit vector and the direct baseline is equal to the perpendicular baseline
    perpendicular_baseline = s * norm(cross(los1 - los2, los1 ./ norm(los1)))
    return perpendicular_baseline
end

baseline_perpendicular (generic function with 1 method)

#### 0) Load relevant data

##### Paths

In [4]:
dem_path = "/Users/eyu/local_data/data/srtm_38_01/srtm_38_01.tif"
slc1_safe_path = "/Users/eyu/local_data/data/phase_bug/BB/S1B_IW_SLC__1SDV_20170408T053951_20170408T054019_005065_008DBC_AEEF.SAFE"
slc2_safe_path = "/Users/eyu/local_data/data/phase_bug/BB/S1B_IW_SLC__1SDV_20170420T053952_20170420T054019_005240_0092C6_3820.SAFE";

##### Load SLC1

In [5]:
slc1_data_path, slc1_meta_path, slc1_calibration_path = Load.slc_paths(slc1_safe_path, "VV", 3);
slc1_meta = Load.slc_meta(slc1_meta_path);
slc1_pod = Load.precise_orbit(Load.pod_path(slc1_meta["t_0"], slc1_meta["mission_id"],
                        "/Users/eyu/local_data/data/phase_bug/POD"), slc1_meta["t_0"]);
#slc1_start_time, slc1_stop_time = UnitTest.meta_start_datetime(slc1_meta_path);


##### Load SLC2

In [6]:
slc2_data_path, slc2_meta_path, slc2_calibration_path = Load.slc_paths(slc2_safe_path, "VV", 3);
slc2_meta = Load.slc_meta(slc2_meta_path);
slc2_pod = Load.precise_orbit(Load.pod_path(slc2_meta["t_0"], slc2_meta["mission_id"],
                        "/Users/eyu/local_data/data/phase_bug/POD"), slc2_meta["t_0"]);

##### Load DEM

In [7]:
padding = 0;
burst_middle = Int((slc1_meta["lines_per_burst"] * slc1_meta["burst_count"]) / 2);
swath_middle = Int((slc1_meta["samples_per_burst"]) / 2);
mosaic_view = [(burst_middle):(burst_middle), swath_middle:swath_middle];  # in mosaic geometry
#mosaic_view = [2104:2104, 12665:12665];  # in mosaic geometry

interest_view = mosaic2normal_view(mosaic_view, slc1_meta);  # in raw geometry

In [9]:
dem = dem_subset(dem_path, slc1_meta, interest_view; nan_fill= 0, padding=[180,180]);

proj_create: init=epsg:/init=IGNF: syntax not supported in non-PROJ4 emulation mode


#### Compute perpendicular baseline

In [26]:
baseline_perpendicular(interest_view, dem, [slc1_meta, slc2_meta], [slc1_pod, slc2_pod])

-48.624954511700345

In [42]:
days_between_acquisitions(extract_datetime(slc1_safe_path), extract_datetime(slc2_safe_path))

-12

# Step by step solution:

#### Get line, sample, height from point of interest

In [31]:
line1, sample1 = Misc.flatten(interest_view[1], interest_view[2])  # line, sample. Needed input structure for to_lat_lon
height = get_elevation(interest_view, dem, slc1_meta, slc1_pod)

68.92261268269999

#### Compute the corresponding lat lon coordinates, and convert to cartesian coordinates

In [32]:
lat, lon = Geometry.to_lat_lon(hcat(line1, sample1), height, slc1_pod[1], slc1_pod[2], slc1_meta)
pixel_position = Geometry.ellipsoid2xyz(deg2rad(lat), deg2rad(lon), height...)

#### Compute the satellite position at the azimuth time corresponding to the line of interest

In [33]:
azimuth_time = slc1_meta["t_start"] + (line[1] - 1) * 1 / slc1_meta["azimuth_frequency"]
state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(slc1_pod[1], slc1_pod[2], slc1_meta["t_start"], slc1_meta["t_stop"]);
tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
sat1_position, sat1_velocity = tmp[1:3], tmp[4:6];

([3.94765e6, 1.17327e6, 5.74427e6], [6285.76, -359.364, -4236.19])

#### Compute line of sight 1

In [35]:
LOS1 = sat1_position .- pixel_position

3-element Array{Float64,1}:
 376165.2779512401 
 691432.9649579779 
 499507.38678614516

#### Compute line sample in slc2 geometry from the lat, lon point

In [ ]:
line2, sample2 = to_line_sample(hcat(lat, lon), height, slc2_pod[1], slc2_pod[2], slc2_meta)

#### Compute the satellite position at the azimuth time corresponding to the line of interest

In [37]:
azimuth_time = slc2_meta["t_start"] + (line2[1] - 1) * 1 / slc2_meta["azimuth_frequency"]
state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(slc2_pod[1], slc2_pod[2], slc2_meta["t_start"], slc2_meta["t_stop"]);
tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
sat2_position, sat2_velocity = tmp[1:3], tmp[4:6];

([3.94765e6, 1.17334e6, 5.74426e6], [6285.77, -359.421, -4236.19])

#### Compute line of sight 2

In [39]:
LOS2 = sat2_position .- pixel_position

3-element Array{Float64,1}:
 376166.50496910745
 691495.0362815189 
 499495.7727967473 

#### Calculate the perpenducular baseline.
The sign keeps sign across many baseline computations consistent wrt to a given master. The norm of the cross product is the area of the parallelogram spanned by the direct baseline btw slc1 and slc2 and the unit vector of slc1. Hence the area only depends on length of perpendicular baseline btw slc 1 and slc2.

In [40]:
s = sign(norm(LOS1) - norm(LOS2));

In [41]:
bperp = s * norm(cross(LOS1 - LOS2, LOS1 ./ norm(LOS1)))

-48.624954511700345

#### The days btw the acquisitions

In [212]:
days_between_acquisitions(extract_datetime(slc1_safe_path), extract_datetime(slc2_safe_path))

-12

1

In [193]:
line, sample = Misc.flatten(interest_view[1], interest_view[2])  # line, sample. Needed input structure for to_lat_lon

([8530], [12665])

In [194]:
height = get_elevation(interest_view, dem, slc1_meta, slc1_pod)

1-element Array{Float64,1}:
 0.0

In [195]:
function line_of_sight_point(line_sample, height, meta, pod)

    sat1_position, sat1_velocity = satellite_orbit(line_sample[1], meta, pod)
    los = sat1_position .- pixel_position
    return los
end

1×2 Array{Float64,2}:
 55.6852  7.68356

(3.5714867991038603e6, 481840.1801384752, 5.24476468370043e6)

([3.94765e6, 1.17327e6, 5.74427e6], [6285.76, -359.364, -4236.19])

3-element Array{Float64,1}:
 376165.2779512401 
 691432.9649579779 
 499507.38678614516

In [202]:
line2, sample2 = to_line_sample(hcat(lat, lon), height, slc2_pod[1], slc2_pod[2], slc2_meta)

1×2 Array{Float64,2}:
 8530.25  12682.3

69.53920089416022

In [204]:
azimuth_time = slc2_meta["t_start"] + (line2[1] - 1) * 1 / slc2_meta["azimuth_frequency"]
state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(slc2_pod[1], slc2_pod[2], slc2_meta["t_start"], slc2_meta["t_stop"]);
tmp = Geometry.polyval_state_vectors(state_vectors_poly, azimuth_time, state_vectors_mean, state_vectors_std)
sat2_position, sat2_velocity = tmp[1:3], tmp[4:6]

([3.94765e6, 1.17334e6, 5.74426e6], [6285.77, -359.421, -4236.19])

In [206]:
LOS2 = sat2_position .- pixel1_position

3-element Array{Float64,1}:
 376166.50496910745
 691495.0362815189 
 499495.7727967473 

In [219]:
s = sign(norm(LOS1) - norm(LOS2));

In [220]:
bperp = s * norm(cross(LOS1 - LOS2, LOS1 ./ norm(LOS1)))

-48.624954511700345

get_elevation (generic function with 1 method)

In [252]:
function get_satellite_pos_vel(line, pod, meta)
    c = 299792458
    t_start = meta["t_start"]
    t_stop = meta["t_stop"]
    inv_azimuth_frequency =  1 / meta["azimuth_frequency"]
    state_vectors = pod[1]
    time_state_vectors = pod[2];

    state_vectors_poly, state_vectors_mean, state_vectors_std = Geometry.satellite_trajectory(state_vectors, time_state_vectors, t_start, t_stop); 

    time =  t_start + (line - 1) * inv_azimuth_frequency
    state_vectors_0 = Geometry.polyval_state_vectors(state_vectors_poly, time, state_vectors_mean, state_vectors_std)
    satellite_position = state_vectors_0[1:3]
    satellite_velocity = state_vectors_0[4:6]

    return satellite_position, satellite_velocity
end

function get_pixel_position(sample, elevation, satellite_position, satellite_velocity, meta)
    theta_0 = sign_angle * abs(slc1_meta["incidence_angle_mid"]*pi/180)
    r_near =  meta["slant_range_time"]  * c / 2
    range_pixel_spacing =  c / (2 * meta["range_sampling_rate"])

    range = r_near + (sample - 1) * range_pixel_spacing

    pixel_position_0 = Geometry.ellipsoid_intersect(satellite_position, Geometry.approx_line_of_sight(satellite_position, satellite_velocity, theta_0))
    pixel_position = Geometry.solve_radar(range, elevation, pixel_position_0, satellite_position, satellite_velocity)
    return pixel_position
end

get_pixel_position (generic function with 1 method)

In [253]:
elevation = get_elevation(interest_view, dem, slc1_meta, slc1_pod)

1-element Array{Float64,1}:
 0.0

In [254]:
line, sample = Misc.flatten(interest_view[1], interest_view[2])  # line, sample. Needed input structure for to_lat_lon


([2286], [12665])

In [255]:
satellite_position, satellite_velocity = get_satellite_pos_vel(line..., slc1_pod, slc1_meta)

([3.86661e6, 1.1777e6, 5.79811e6], [6342.58, -330.643, -4152.61])

In [256]:
pixel_position = get_pixel_position(sample..., elevation..., satellite_position, satellite_velocity, slc1_meta)

3-element Array{Float64,1}:
      3.499453703803955e6
 485928.2519566412       
      5.292405763882056e6

In [257]:
LOS1 = satellite_position - pixel_position

3-element Array{Float64,1}:
 367155.33699074807
 691773.1556225147 
 505701.8395259371 

In [258]:
elevation = get_elevation(interest_view, dem, slc2_meta, slc2_pod);

In [259]:
line, sample = Misc.flatten(interest_view[1], interest_view[2]);  # line, sample. Needed input structure for to_lat_lon


In [260]:
satellite_position, satellite_velocity = get_satellite_pos_vel(line..., slc2_pod, slc2_meta);

In [261]:
pixel_position = get_pixel_position(sample..., elevation..., satellite_position, satellite_velocity, slc2_meta);

In [262]:
LOS2 = satellite_position - pixel_position

3-element Array{Float64,1}:
 367156.3318909472
 691777.1691301952
 505695.6268777065

In [263]:
s = sign(norm(LOS1) - norm(LOS2));

In [264]:
bperp = s * norm(cross(LOS1 - LOS2, LOS1 ./ norm(LOS1)))

-7.462912859199071

In [ ]:

# Get latitude and longitude
lat_lon = to_lat_lon(hcat(master_line, master_sample), lut["heights"],
precise_orbit[1]...,meta[1])
lut["latitude"] = lat_lon[:,1]
lut["longitude"] = lat_lon[:,2]
@assert sum(isnan.(lut["latitude"])) == 0
@assert sum(isnan.(lut["longitude"])) == 0

In [200]:
lat_lon

1×2 Array{Float64,2}:
 55.6214  8.36889

### SLC1
#### 1) Udregn koordinaterne (lat, lon) for et udvalgt (sample,line) punkt baseret på SLC 1 parametre. 
Brug det miderste i swath

In [10]:
state_vectors = slc1_pod[1]
time_state_vectors = slc1_pod[2];

In [43]:
## FIRST PART OF LUT, INTERPOLATE HEIGHTS
slc1_line_sample = hcat(Misc.flatten(interest_view[1], interest_view[2])...)  # line, sample. Needed input structure for to_lat_lon

lat_lon, point, x = to_lat_lon_single_point_version(slc1_line_sample, heights, state_vectors, time_state_vectors, slc1_meta);

In [44]:
lat_lon, point, x

([55.6214 8.36886], (0.9707764316805281, 0.14606410755674515, 5.999992641620338), [3.57128e6, 525377.0, 5.24076e6])

In [25]:
n_1 = length(dem[1])
n_2 = length(dem[2])

index1_grid =  Array{eltype(dem[1])}(undef, n_1, n_2)
index2_grid =  Array{eltype(dem[2])}(undef, n_1, n_2)

for i in 1:n_2
    index1_grid[:,i] .= dem[1]
end

for j in 1:n_1
    index2_grid[j,:] .= dem[2]
end

In [33]:
test = scipy_interp.griddata(hcat(vec(index1_grid), vec(index2_grid)), vec(dem[3]), hcat(55.70, 8.36));
#Misc.interp(index1_grid, index2_grid, dem[3], lat_lon...)

##### interpolate to above lat_lon point using dem[1] and dem[2]

#### 2) Konverter (lat, lon)-punktet til kartesiske koordinater
punktet på jorden / den ene ende af vores LOS vektor

In [ ]:
[x,y,z] = Geometry.ellipsoid2xyz(lat, lon, height; semi_major_axis=6378137.,
                       flattening=1/298.257223563)

for i in 1:size(lat_lon)[1]
point_xyz = ellipsoid2xyz(lat_lon[i,1]* deg2rad, lat_lon[i,2]* deg2rad ,height[i])




#### 3) Udregn satellitens position og hastighed fra state vectors tilhørende SLC 1

In [ ]:
state_vectors_poly, state_vectors_mean, state_vectors_std = satellite_trajectory(state_vectors,time_state_vectors, t_start, t_stop; poly_degree=4, max_time_margin=80.)

#### 3a) Find satellitens interpolerede position (polyval)

In [ ]:
state_vectors = polyval_state_vectors(state_vectors_poly, trial_time, state_vectors_mean, state_vectors_std)
        trial_sat_position = state_vectors[1:3]
        line_of_sight = point_xyz .- trial_sat_position

#### 4) Udregn LOS1 ved at trække xyz punktet fra satellitens interpolerede position

### SLC2
#### 5) Udregn de (sample, line) koordinater svarende til (lat,lon,height) punktet fra SLC1
Brug SLC2 parametre

#### 6) Udregn satellitens position og hastighed fra state vectors tilhørende SLC 2

#### 6a) Find satellitens interpolerede position (polyval)

#### 7) Udregn LOS2 ved at trække xyz punktet fra satellitens interpolerede position

#### 8)
sign = sign(norm(LOS1 - LOS2)
baseline = LOS1 - LOS2
LOS1_unit_vector = LOS1 ./ norm(LOS1)
baseline_perpendicular = sign * norm(cross(baseline, LOS1_norm))

### Trash

In [87]:
function dem_test(path, lat_lon_window; nan_fill= NaN, padding=[0,0],nan_value =-32768)

    dem_annotations = rasterio.open(path)
    # find the corresponding slc corner in row, col of the dem and add padding
    (max_row, min_col) = dem_annotations.index(lat_lon_window[2][1], lat_lon_window[1][1])
    (min_row, max_col) = dem_annotations.index(lat_lon_window[2][2], lat_lon_window[1][2]);


    # make intervals with padding for .read's window function
    min_row = min_row - padding[1]
    max_row = max_row + padding[1]
    min_col = min_col - padding[2]
    max_col = max_col + padding[2]


    if min_row < 0
        min_row = 0
        println("Warning min row are not in the picture.")
    end

    if max_row > dem_annotations.height
        max_row = dem_annotations.height
        println("Warning max row are not in the picture.")
    end

    if min_col < 0
        min_col = 0
        println("Warning min column are not in the picture.")
    end

    if max_col > dem_annotations.width
        max_col = dem_annotations.width
        println("Warning max column are not in the picture.")
    end


    row_interval = (min_row , max_row)
    col_interval = (min_col , max_col)

    # load subset of dem
    if padding[1] == 0 && padding[2] == 0
        dem_data = dem_annotations.read(1)
    else
        dem_data = dem_annotations.read(1, window=(row_interval, col_interval))
    end
    #dem_data = dem_annotations.read(1, window=(row_interval, col_interval))
    
    dem_data[dem_data .== (nan_value)] .= nan_fill;

    #dem_view = [row_interval[1]:row_interval[2], col_interval[1]:col_interval[2]]
    transform = dem_annotations.get_transform()

    rows = collect(1:dem_annotations.height).-1;
    columns = collect(1:dem_annotations.width).-1;
    lon = transform[1] .+ rows .* transform[2];
    lat  = transform[4] .+ columns .* transform[6];

    #dem = dem[index1,index2];
    lat = lat[row_interval[1]:row_interval[2]-1]
    lon = lon[col_interval[1]:col_interval[2]-1];

    # return subset of dem and dem_view
    return lat, lon, dem_data
end

dem_test (generic function with 1 method)